In [ ]:
from pandas import DataFrame, read_csv
from collections import defaultdict
from scipy import stats
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
import json
with open('data/centroids.json', 'r') as infile:
    centroids = json.load(infile)
with open('data/labels.json', 'r') as infile:
    labels = json.load(infile)
data_csv = 'data/normalized_points.csv'
data_points = pd.read_csv(data_csv,header=None)

In [ ]:
#Reminder: attributes start their numbering with 2, not 0
plots_dd = defaultdict(lambda:[])
for i, row in data_points.iterrows():
    plots_dd[row[0]].append((row[1],labels[i]))
for i in plots_dd:
    plots_dd[i].sort()
data_points.set_index([0,1],drop=True,inplace=True)

In [ ]:
markov_df = pd.DataFrame(0.0, index=np.arange(data_points.shape[1]), columns=np.arange(data_points.shape[1]))
for i in plots_dd:
    prev_y = -1
    prev_s = -1
    for elem in plots_dd[i]:
        if (prev_y>-1):
            markov_df.iloc[prev_s][elem[1]] += 1 - 0.5**(1/(elem[0]-prev_y))
        prev_y = elem[0]
        prev_s = elem[1]
r_sum = markov_df.sum(axis=1)
r_sum = r_sum.replace(0.0,1.0)
markov_df = markov_df.truediv(r_sum,axis=0)

In [ ]:
#mc: DataFrame representing the Markov Chain
def fitness(mc):
    score = 0.0
    num_cases = 0
    for i in plots_dd:
        prev_y = -1
        prev_s = -1
        for elem in plots_dd[i]:
            if (prev_y>-1):
                num_cases+=1
                dist = []
                cur_v = mc.xs(elem[1])
                for c in centroids:
                    dist.append(np.linalg.norm(cur_v-c))
                score += np.sum(((mc**(elem[0]-prev_y)).xs(prev_s))*dist)
            prev_y = elem[0]
            prev_s = elem[1]
    return score/num_cases

In [ ]:
#The fitness function takes 11 seconds to compute, it has to be worked on
fitness(markov_df)